In [ ]:
!pip install turicreate
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install sklearn

In [ ]:
import turicreate as tc

In [ ]:
sf = tc.SFrame.read_csv('ratings.csv', verbose = False)
sf = sf.remove_column('rating_id') #remove the db id
sf2 = tc.SFrame.read_csv('trails.csv', verbose = False)
sf2 = sf2.select_columns(['trail_id', 'name'])
sf = sf.join(sf2, on='trail_id', how='inner')
sf = sf.rename({'trail_id': 'item_id'}) #rinomiano il trail_id in item_id in maniera tale da conformarci con la libreria Turicreate
sf = sf.rename({'name': 'item_name'}) #rinomiano il trail_id in item_id in maniera tale da conformarci con la libreria Turicreate
sf2 = sf2.rename({'trail_id': 'item_id'})
sf 

## Item Based Collaborative Filtering - Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare l'item based recommender system sul dataset, inoltre proviamo a buildare il modello utilizzando dei parametri differenti per testare l'efficacia del sistema

In [ ]:
item_based_cf_model = tc.recommender.item_similarity_recommender.create(sf, user_id = 'user_id' , item_id = 'item_id' , 
                                                          target = 'user_rating' , user_data = None ,
                                                          item_data = None , nearest_items = None , similarity_type = 'jaccard' ,
                                                          threshold = 0.001, verbose = False)
    
ibcf_similarity = item_based_cf_model.get_similar_items(k=3, verbose = False)
ibcf_similarity = ibcf_similarity.join(sf2, on='item_id', how='inner')
ibcf_similarity = ibcf_similarity.join(sf2, on={'similar':'item_id'}, how='inner')
ibcf_similarity = ibcf_similarity.rename({'name': 'item_name', 'name.1': 'similar_name'})
ibcf_similarity.print_rows(num_rows=20, num_columns=6, max_row_width = 150)
ibcf_recommend = item_based_cf_model.recommend(k=3, verbose = False)
ibcf_recommend = ibcf_recommend.join(sf2, on='item_id', how='inner')
ibcf_recommend.print_rows(num_rows=20, num_columns=5, max_row_width = 150)


## User Based Collaborative Filtering - Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare l'user based recommender system sul dataset, utilizzeremo la stessa tipologia di modello utilizzata in precedenza ma con i parametri user_id e item_id invertiti come suggerisce la documentazione. Inoltre, come fatto nel caso precedente,  testiamo anche diversi parametri per valutare il sistema con l'efficacia migliore

In [ ]:
user_based_cf_model = tc.recommender.item_similarity_recommender.create(sf, user_id = 'item_id' , item_id = 'user_id' , 
                                                          target = 'user_rating' , user_data = None ,
                                                          item_data = None , nearest_items = None , similarity_type = 'cosine' ,
                                                          threshold = 0.001, verbose = False)
    
ubcf_similarity = user_based_cf_model.get_similar_items(k=5, verbose = False)
ubcf_similarity.print_rows(num_rows=20, num_columns=4)
ubcf_recommendation = user_based_cf_model.recommend(k=5, verbose = False)
ubcf_recommendation = ubcf_recommendation.join(sf2, on='item_id', how='inner')
ubcf_recommendation = ubcf_recommendation.sort([('user_id', False), ('score', False)])
ubcf_recommendation = ubcf_recommendation.remove_column('rank')
ubcf_recommendation.print_rows(num_rows=20, num_columns=5, max_row_width = 150)


## Matrix Factorization Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare la matrix factorization sul dataset, alternativa il cui approccio alla base risulta simile al Collaborative Filtering. Proviamo anche qui a far runnare il modello con diversi parametri

In [ ]:
matrix_factorization_model = tc.ranking_factorization_recommender.create(sf, target = 'user_rating' ,
                                                         ranking_regularization = 0.1 ,
                                                         unobserved_rating_value = 1,
                                                         max_iterations = 50, solver = "sgd", 
                                                         verbose = False)
matrix_similarity = matrix_factorization_model.get_similar_items(k = 5, verbose = False)
matrix_similarity = matrix_similarity.join(sf2, on='item_id', how='inner')
matrix_similarity = matrix_similarity.join(sf2, on={'similar':'item_id'}, how='inner')
matrix_similarity = matrix_similarity.rename({'name': 'item_name', 'name.1': 'similar_name'})
matrix_similarity.print_rows(num_rows=20, num_columns=6, max_row_width=150)
matrix_recommendation = matrix_factorization_model.recommend(k = 5, verbose = False)
matrix_recommendation = matrix_recommendation.join(sf2, on='item_id', how='inner')
matrix_recommendation.print_rows(num_rows=20, num_columns=5, max_row_width=150)

## Content Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare un Content Based Recommender sul dataset, in particolare lo faremo sul file trails.csv che contiene tutte le feature piu' importanti di un percorso (circa 60 in totale)

In [ ]:
trails = tc.SFrame.read_csv('trails.csv', verbose = False)
trails = trails.rename({'trail_id': 'item_id'}) #rinomiano il trail_id in item_id in maniera tale da conformarci con la libreria 
content_based_model = tc.recommender.item_content_recommender.create(item_data = trails, item_id="item_id", 
                                                                     observation_data = sf, user_id="user_id", 
                                                                     target="user_rating", verbose = False)
content_based_similiarity = content_based_model.get_similar_items(verbose = False)
content_based_similiarity = content_based_similiarity.join(sf2, on='item_id', how='inner')
content_based_similiarity = content_based_similiarity.join(sf2, on={'similar':'item_id'}, how='inner')
content_based_similiarity = content_based_similiarity.rename({'name': 'item_name', 'name.1': 'similar_name'})
content_based_similiarity.print_rows(num_rows=20, num_columns=6, max_row_width=150)
content_based_recommend = content_based_model.recommend(verbose = False)
content_based_recommend = content_based_recommend.join(sf2, on='item_id', how='inner')
content_based_recommend.print_rows(num_rows=20, num_columns=5, max_row_width=150)

## Popular Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare il Recommender system piu' semplice: il popular based. Le raccomandazioni fornite sono utilii per superare il problema del Cold Start e per spingere l'utente a recensire piu' item possibili al fine di ottenere in futuro delle raccomandazioni che siano il piu' possibile pertinenti.

In [ ]:
popular_based_model = tc.popularity_recommender.create(sf, target = 'user_rating', verbose = False)
popresult = popular_based_model.recommend()
popresult = popresult.join(sf2, on='item_id', how='inner')
popresult.print_rows(num_rows=20, num_columns=5)